In [2]:
# Code adapted from UIUC PS 590: Images and Text as Data
# by Prof. Nora Webb Williams. She adapted from adapted 
# from Diyi Yang and crew at Georgia Tech.
import ktrain
import pandas as pd

import ktrain
from ktrain import text as ktext
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score

df = pd.read_csv("../TwitterData/prelim_dataset.csv")
text = df['text'].to_list()
# TODO: Text pre-processing?
labels = df['Islamophobic?'].to_list()
target_name = ['Not islamophobic', 'Islamophobic']
# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(text, labels, random_state = 0)
# Preprocess data
trn, val, preproc = ktext.texts_from_array(x_train=X_train, y_train=y_train,
                                          x_test=X_test, y_test=y_test,
                                          class_names = target_name,
                                          preprocess_mode='distilbert',
                                          maxlen=128)
# Building a model and a learner
model = ktext.text_classifier('distilbert', train_data=trn, preproc=preproc)
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=16)
# Train the model
learner.fit_onecycle(3e-5, 4)

# Predict on test set
p = ktrain.get_predictor(model, preproc)
y_pred = p.predict(X_test)

y_pred_num = []
for pred in y_pred:
    rel_index = target_name.index(pred)
    y_pred_num.append(rel_index)

print(metrics.classification_report(y_test, y_pred_num))
print("Macro:", f1_score(y_test, y_pred_num, average='macro'))
print("Micro:", f1_score(y_test, y_pred_num, average='micro'))

preprocessing train...
language: en
train sequence lengths:
	mean : 37
	95percentile : 100
	99percentile : 216


C:\Users\emilysallens\AppData\Roaming\Python\Python39\site-packages\ktrain\utils.py:641: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  warnings.warn('class_names implies classification but targets array contains float(s) instead of integers or strings')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 36
	95percentile : 104
	99percentile : 193


task: text classification
Is Multi-Label? False
maxlen is 128
done.


begin training using onecycle policy with max lr of 3e-05...
Epoch 1/4
37/37 [==============================] - 334s 8s/step - loss: 0.5767 - accuracy: 0.7551 - val_loss: 0.4372 - val_accuracy: 0.8283
Epoch 2/4
37/37 [==============================] - 271s 7s/step - loss: 0.4220 - accuracy: 0.8243 - val_loss: 0.3867 - val_accuracy: 0.8283
Epoch 3/4
37/37 [==============================] - 408s 11s/step - loss: 0.3637 - accuracy: 0.8243 - val_loss: 0.3694 - val_accuracy: 0.8283
Epoch 4/4
7/7 [==============================] - 67s 8s/step
              precision    recall  f1-score   support

         0.0       0.85      0.99      0.91       164
         1.0       0.71      0.15      0.24        34

    accuracy                           0.84       198
   macro avg       0.78      0.57      0.58       198
weighted avg       0.83      0.84      0.80       198



In [4]:
from sklearn.metrics import f1_score

print(metrics.classification_report(y_test, y_pred_num))
print("Macro:", f1_score(y_test, y_pred_num, average='macro'))
print("Micro:", f1_score(y_test, y_pred_num, average='micro'))

              precision    recall  f1-score   support

         0.0       0.85      0.99      0.91       164
         1.0       0.71      0.15      0.24        34

    accuracy                           0.84       198
   macro avg       0.78      0.57      0.58       198
weighted avg       0.83      0.84      0.80       198

Macro: 0.5782892476812093
Micro: 0.8434343434343435
